In [91]:
# Study partner is Nicole Lund.  Worked on Pseudo code together and hiding api keys.
# Received assistance from instructor via office hours and tutoring sessions.
# The starter code and pseudo code for this challenge was provided and can be reviewed in data folder 
# Resources: past class assignments and Google searches


# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [92]:
!pip install gmaps

In [93]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

# Incorporated citipy to determine city based on latitude and longitude
from citipy import citipy
if g_key == "YOUR KEY HERE!":
    import sys
    sys.path.append(r"C:\Users\Brad\Documents\GitHub\Untracked_Files")
    from api_keys_ATN import g_key
# print(g_key)

# Output File (CSV)
output_data_file = "output_data/cities.csv"

# Range of latitudes and longitudes
lat_range = (-90, 90)
lng_range = (-180, 180)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [94]:
cities_loop_pd=pd.read_csv('cities_loop.csv')
cities_loop_pd

,city,latitude,longitude,temperature,humidity,clouds,wind speed,date,country
0,hithadhoo,-0.6000,73.0833,28.17,73,67,3.21,1619117764,MV
1,cherskiy,68.7500,161.3000,-4.83,96,100,6.00,1619117765,RU
2,vostok,46.4856,135.8833,6.60,54,100,1.76,1619117765,RU
3,busselton,-33.6500,115.3333,15.00,77,93,1.79,1619117552,AU
4,lalin,42.7167,-8.1167,16.67,67,20,2.57,1619117766,ES
...,...,...,...,...,...,...,...,...,...
557,piranshahr,36.7010,45.1413,14.88,58,40,2.77,1619117964,IR
558,troitskoye,52.9821,84.6757,-2.47,86,23,4.38,1619117965,RU
559,tiznit,29.5833,-9.5000,16.63,60,8,4.31,1619117965,MA
560,formosa,-26.1775,-58.1781,22.00,78,75,9.26,1619117838,AR


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [95]:
gmaps.configure(api_key=g_key)

In [96]:
# Store 'Lat' and 'Lng' into  locations 
locations = cities_loop_pd[["latitude", "longitude"]].astype(float)

humidity_rate = cities_loop_pd["humidity"].astype(float)

fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity_rate, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [97]:
# Narrow down the DataFrame to find your ideal weather condition.
#   * A max temperature lower than 80 degrees but higher than 30.
#   * Wind speed less than 30 mph.
#   * Cloudiness 10

Ideal_weather=cities_loop_pd.loc[(cities_loop_pd['temperature'] < 80) & (cities_loop_pd['temperature'] > 30) & (cities_loop_pd['wind speed'] < 30) & (cities_loop_pd['clouds'] <10)]
Ideal_weather


,city,latitude,longitude,temperature,humidity,clouds,wind speed,date,country
195,doha,25.2867,51.5333,30.56,48,0,1.03,1619117834,QA
271,taoudenni,22.6783,-3.9836,36.19,7,0,3.10,1619117860,ML
349,paita,-5.0892,-81.1144,31.00,48,0,7.72,1619117888,PE
356,atar,20.5169,-13.0499,36.00,23,0,1.03,1619117891,MR
363,buraydah,26.3260,43.9750,31.00,18,0,3.60,1619117893,SA
408,tessalit,20.1986,1.0114,31.87,6,1,4.63,1619117909,ML
414,riyadh,24.6877,46.7219,32.00,13,0,2.06,1619117865,SA
467,bafata,12.1667,-14.7500,35.68,21,9,5.10,1619117931,GW
509,adrar,20.5022,-10.0711,31.00,8,0,4.33,1619117947,MR
527,santa marta,11.2408,-74.1990,31.67,63,8,0.89,1619117954,CO


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [98]:
# new variable name
hotel_df=Ideal_weather.copy()

# Create a new column
hotel_df['Hotel Name'] = ""
hotel_df

,city,latitude,longitude,temperature,humidity,clouds,wind speed,date,country,Hotel Name
195,doha,25.2867,51.5333,30.56,48,0,1.03,1619117834,QA,
271,taoudenni,22.6783,-3.9836,36.19,7,0,3.10,1619117860,ML,
349,paita,-5.0892,-81.1144,31.00,48,0,7.72,1619117888,PE,
356,atar,20.5169,-13.0499,36.00,23,0,1.03,1619117891,MR,
363,buraydah,26.3260,43.9750,31.00,18,0,3.60,1619117893,SA,
408,tessalit,20.1986,1.0114,31.87,6,1,4.63,1619117909,ML,
414,riyadh,24.6877,46.7219,32.00,13,0,2.06,1619117865,SA,
467,bafata,12.1667,-14.7500,35.68,21,9,5.10,1619117931,GW,
509,adrar,20.5022,-10.0711,31.00,8,0,4.33,1619117947,MR,
527,santa marta,11.2408,-74.1990,31.67,63,8,0.89,1619117954,CO,


In [99]:
# setting paramaters to search
target_radius = 5000
target_type = "Hotel"
keyword= "hotel"
key="g_key"

# set up a parameters dictionary
params = {
    "radius": target_radius,
    "type": target_type,
    "key": g_key,
    "keyword": keyword
 
}
# set forloop
for index, row in hotel_df.iterrows():
    lat=row['latitude']
    lon=row['longitude']

    params['location'] =f"{lat},{lon}"
    
    # base url
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # run a request using our params dictionary
    response = requests.get(base_url, params=params)

    places_data = response.json()
    try:
        hotel_df.loc[index, "Hotel Name"]=places_data['results'][0]['name']
        print(places_data['results'][0]['name'])
    except:
        print("Hotel not Found")
        hotel_df.loc[index, "Hotel Name"]="Hotel not Found"

JW Marriott Marquis City Center Doha
Hotel not Found
Hotel Marina Del Bay
les toiles maures
فندق تاورز HOTELTOWERS
Hotel not Found
Park Inn by Radisson, Riyadh
Hotel not Found
Hotel not Found
Hilton Garden Inn Santa Marta, Colombia


In [100]:
hotel_df


,city,latitude,longitude,temperature,humidity,clouds,wind speed,date,country,Hotel Name
195,doha,25.2867,51.5333,30.56,48,0,1.03,1619117834,QA,JW Marriott Marquis City Center Doha
271,taoudenni,22.6783,-3.9836,36.19,7,0,3.10,1619117860,ML,Hotel not Found
349,paita,-5.0892,-81.1144,31.00,48,0,7.72,1619117888,PE,Hotel Marina Del Bay
356,atar,20.5169,-13.0499,36.00,23,0,1.03,1619117891,MR,les toiles maures
363,buraydah,26.3260,43.9750,31.00,18,0,3.60,1619117893,SA,فندق تاورز HOTELTOWERS
408,tessalit,20.1986,1.0114,31.87,6,1,4.63,1619117909,ML,Hotel not Found
414,riyadh,24.6877,46.7219,32.00,13,0,2.06,1619117865,SA,"Park Inn by Radisson, Riyadh"
467,bafata,12.1667,-14.7500,35.68,21,9,5.10,1619117931,GW,Hotel not Found
509,adrar,20.5022,-10.0711,31.00,8,0,4.33,1619117947,MR,Hotel not Found
527,santa marta,11.2408,-74.1990,31.67,63,8,0.89,1619117954,CO,"Hilton Garden Inn Santa Marta, Colombia"


In [104]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{city}</dd>
<dt>Country</dt><dd>{country}</dd>
</dl>
"""
#***I had to use lowercase city and country in order for the code to work***

# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["latitude", "longitude"]]

In [107]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(
    locations,info_box_content=hotel_info)

fig = gmaps.figure()
fig.add_layer(marker_layer)

fig

# Display figure


Figure(layout=FigureLayout(height='420px'))